# Batch Normalization

Teinar modelos profundos e fazê-los convergir numa quantidade razoável de tempo pode ser   uma tarefa complicada.
Nesta prática, descrevemos e usaremos o [*batch normalization*](https://arxiv.org/abs/1502.03167) (BN), uma técnica popular e eficaz capaz de acelerar a convergência de redes profundas que, juntamente com [blocos residuais](https://arxiv.org/abs/1512.03385), nos permitiu treinar redes com mais de 100 (até mesmo 1000) camadas.

Primeiro, vamos rever alguns dos desafios práticos ao treinar redes profundas.

1. O pré-processamento de dados geralmente se prova crucial modelagem estatística. Como falado anteriormente, num geral, se padroniza a entrada para ter uma média *zero* e variância de *um*. Padronizar os dados de entrada normalmente torna mais fácil treinar modelos profundos, pois os parâmetros estão, *a priori*, em uma escala similar.  
1. Para reles MLP ou CNN, enquanto treinamos o modelo, as ativações em camadas intermediárias da rede podem assumir diferentes ordens de magnitude. Os autores de [*batch normalization*](https://arxiv.org/abs/1502.03167) postulou que esta diferença na distribuição das ativações poderia dificultar a convergência da rede. Intuitivamente, poderíamos conjeturar que, se camada tem valores de ativação que são 100x que de outra camada, poderíamos precisa ajustar as taxas de aprendizagem de forma adaptável por camada (ou mesmo para neurônios dentro de uma mesma camada).
1. Redes mais profundas são complexas e propensas à *overfitting*. Isso significa que a regularização se torna mais importante. Empiricamente, notamos que mesmo com o *dropout*, os modelos podem cair numa situação de *overfitting*. Neste caso, devemos nos beneficiar de outras heurística de regularização.
 


In [1]:
!pip3 install torch torchvision

In [0]:
import time, os, sys, numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torchsummary import summary


import time, os, sys, numpy as np

# Test if GPU is avaliable, if not, use cpu instead
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids= list(range(n))

In [0]:
def load_data_cifar10(batch_size, resize=None, root=os.path.join(
        '~', '.pytorch', 'datasets', 'fashion-mnist')):
    """Download the Cifar10-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [torchvision.transforms.Resize(resize)]
    transformer += [torchvision.transforms.ToTensor()]
    transformer = torchvision.transforms.Compose(transformer)

    mnist_train = torchvision.datasets.CIFAR10(root=root, train=True,download=True,transform=transformer)
    mnist_test = torchvision.datasets.CIFAR10(root=root, train=False,download=True,transform=transformer)
    num_workers = 0 if sys.platform.startswith('win32') else 4



    train_iter = torch.utils.data.DataLoader(mnist_train,
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test,
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.pytorch', 'datasets', 'fashion-mnist')):
    """Download the Fashion-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [torchvision.transforms.Resize(resize)]
    transformer += [torchvision.transforms.ToTensor()]
    transformer = torchvision.transforms.Compose(transformer)

    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True,download=True,transform=transformer)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False,download=True,transform=transformer)
    num_workers = 0 if sys.platform.startswith('win32') else 4



    train_iter = torch.utils.data.DataLoader(mnist_train,
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test,
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

# funções básicas
def _get_batch(batch):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.type() != features.type():
        labels = labels.type(features.type())
    return (torch.nn.DataParallel(features, device_ids=devices_ids),
            torch.nn.DataParallel(labels, device_ids=devices_ids), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss):
    """Evaluate accuracy of a model on the given data set."""

    acc_sum, n, l = torch.Tensor([0]), 0, 0
    
    with torch.no_grad():
      for X, y in data_iter:
          #y = y.astype('float32')
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y).sum()
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
          n += y.size()[0]

    return acc_sum.item() / n, l.item() / len(data_iter)
  
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss,
                   num_epochs):
    print('training on', device)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            trainer.zero_grad()
            l = loss(y_hat, y).sum()
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size()[0]
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)
        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss, 
                 test_acc, time.time() - start))

## Detalhes Técnicos

Em 2015, uma heurística inteligente chamada [*batch normalization*](https://arxiv.org/abs/1502.03167) provou ser imensamente útil para melhorar a confiabilidade e a velocidade de convergência no treinamento de modelos profundos. Em cada iteração de treinamento, o  [*batch normalization*](https://arxiv.org/abs/1502.03167) normaliza as ativações de cada neurônio da camada oculta (em cada camada onde é aplicada) subtraindo sua média e dividindo pelo seu desvio padrão, estimando ambos com base no mini-batch atual. Note que se o tamanho do batch fosse $1$, não aprenderíamos nada porque durante o treinamento, todos os nó levaria valor $0$. No entanto, com minibatches grandes o suficiente, a abordagem se prova eficaz e estável.

Em poucas palavras, a ideia do [*batch normalization*](https://arxiv.org/abs/1502.03167) é transformar a ativação em uma determinada camada de $\mathbf{x}$ para:

$$\mathrm{BN}(\mathbf{x}) = \mathbf{\gamma} \odot \frac{\mathbf{x} - \hat{\mathbf{\mu}}}{\hat\sigma} + \mathbf{\beta}$$

Aqui, $\hat{\mathbf{\mu}}$ é a estimativa da média e $\hat {\mathbf{\sigma}}$ é a estimativa da variância. O resultado é que as ativações são aproximadamente reescaladas para uma média zero e uma variância unitária. Como podemos notar, as ativações das camadas intermediárias não pode divergir muito pois estamos ativamente redimensionando-as de volta para uma dada ordem de grandeza através de $\mathbf{\mu}$ e $\sigma$. Entretanto, em alguns casos, as ativações pode precisar diferir dos dados padronizados. Para lidar com isso e dar mais liberdade à essa normalização, definimos um coeficiente de escala de coordenadas $\mathbf{\gamma}$ e um offset $\mathbf{\beta}$. Intuitivamente, espera-se que essa normalização nos permita ser mais agressivo ao escolher taxas de aprendizado maiores.

Em princípio, podemos querer usar todos os nossos dados de treinamento para estimar a média e variância. No entanto, as ativações correspondentes a cada exemplo mudar cada vez que atualizamos nosso modelo. Para remediar este problema, o [*batch normalization*](https://arxiv.org/abs/1502.03167) usa apenas o minibatch atual para estimar $\hat{\mathbf {\mu}} $ e $\hat \sigma$. É justamente por esse fato de normalizamos com base apenas no *batch* atual que  o método se chama *batch normalization*. Para indicar qual *minibatch* $\mathcal {B}$ é usado, nós denotamos essas variáveis como $\hat{\mathbf{\mu}}_\mathcal {B}$ e $\hat \sigma_\mathcal {B}$.

$$\hat{\mathbf{\mu}}_\mathcal{B} \leftarrow \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} \mathbf{x}$$

$$\hat{\mathbf{\sigma}}_\mathcal{B}^2 \leftarrow \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} (\mathbf{x} - \mathbf{\mu}_{\mathcal{B}})^2 + \epsilon$$

Observe que adicionamos uma pequena constante $\epsilon > 0$ à estimativa de variância para garantir que nunca acabemos por dividir por zero, mesmo nos casos em que a estimativa de variância pode desaparecer por acidente.
Vamos agora ver, na prática, o uso do [*batch normalization*](https://arxiv.org/abs/1502.03167).

## Camadas de *Batch Normalization*

O [*batch normalization*](https://arxiv.org/abs/1502.03167) para camadas totalmente conectadas (*fully-connected* ou Densas) e camadas convolucionais são ligeiramente diferentes. Isso se deve à dimensionalidade dos dados gerados pelas camadas convolucionais. Os dois casos são discutidos abaixo. Observe que uma das principais diferenças entre a camada de [*batch normalization*](https://arxiv.org/abs/1502.03167) e outras camadas é que a primeira opera em um *minibatch* completo de cada vez (caso contrário, não é possível calcular os parâmetros de média e variância por *batch*).

### Camadas densas

Normalmente, aplicamos a camada de [*batch normalization*](https://arxiv.org/abs/1502.03167) entre a transformação e a função de ativação em uma camada densa. A seguir, denotamos por $\mathbf{u}$ a entrada e por $\mathbf{x} = \mathbf{W}\mathbf{u} + \mathbf{b}$ a saída da transformada linear. Isso produz a seguinte fórmula:

$$\mathbf{y} = \phi(\mathrm{BN}(\mathbf{x})) =  \phi(\mathrm{BN}(\mathbf{W}\mathbf{u} + \mathbf{b}))$$

Lembre-se de que a média e a variância são calculadas no **mesmo** *minibatch* $\mathcal{B}$ no qual a transformação é aplicada. Lembre-se também que o coeficiente $\mathbf{\gamma}$ e o offset $\mathbf{\beta}$ são parâmetros que precisam ser aprendidos. Eles garantem que o efeito do [*batch normalization*](https://arxiv.org/abs/1502.03167) possa ser neutralizado conforme necessário.

### Camadas convolucionais

Para camadas convolucionais, o [*batch normalization*](https://arxiv.org/abs/1502.03167) ocorre após o cálculo da convolução e antes da aplicação da função de ativação. Se a computação de convolução gerar múltiplos canais, realizamos o [*batch normalization*](https://arxiv.org/abs/1502.03167) para **cada** uma das saídas desses canais, que tem parâmetros ($\mathbf{\gamma}$ e $\mathbf{\beta}$) independentes. Suponha que haja exemplos de $m$ no *batch*. Em um único canal, assumimos que a altura e a largura da saída da convolução são $p$ e $q$, respectivamente. Precisamos realizar o [*batch normalization*](https://arxiv.org/abs/1502.03167) para $m \times p \times q$ elementos neste canal simultaneamente. Ao executar o cálculo de padronização para esses elementos, usamos a mesma média e variância. Em outras palavras, usamos as médias e as variâncias dos elementos $m \times p \times q$ neste canal em vez de um por pixel.


## Pytorch e o caso de estudo LeNet-5

Agora vamos implementar a [LeNet-5](https://ieeexplore.ieee.org/document/726791) usando a camada de [*batch normalization*](https://arxiv.org/abs/1502.03167).

Em frameworks modernos, camadas de [*batch normalization*](https://pytorch.org/docs/stable/nn.html#normalization-layers) já vem implementadas e são fáceis de usar.

In [9]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 10, 0.01, 128, 0.0001

# rede baseada na LeNet-5 
net = nn.Sequential(
        nn.Conv2d(in_channels=1,out_channels=6, kernel_size=5, stride=1, padding=0),   # entrada: (b, 1, 32, 32) e saida: (b, 6, 28, 28)
        nn.BatchNorm2d(num_features=6),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2, stride=2, padding=0),     # entrada: (b, 6, 28, 28) e saida: (b, 6, 14, 14)
        nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),  # entrada: (b, 6, 14, 14) e saida: (b, 16, 10, 10)
        nn.BatchNorm2d(num_features=16),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2, stride=2, padding=0),     # entrada: (b, 16, 10, 10) e saida: (b, 16, 5, 5)
        nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1, padding=0), # entrada: (b, 16, 5, 5) e saida: (b, 120, 1, 1)
        nn.BatchNorm2d(num_features=120),
        nn.Tanh(),
        nn.Flatten(),  # lineariza formando um vetor         # entrada: (b, 120, 1, 1) e saida: (b, 120*1*1) = (b, 120)
        nn.Linear(120, 84),
        nn.BatchNorm1d(84),
        nn.Tanh(),
        nn.Linear(84, 10))


# Sending model to device
net.to(device)
print(summary(net,(1,32,32))) # visualize number of parameters' net, output of each layer and total mega bytes necessary for forward pass
                                # and stored weights. 

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=32)

# trainer do gluon
trainer = optim.Adam(net.parameters(), lr=lr, weight_decay=wd_lambda)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
                num_epochs)

0it [00:00, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
       BatchNorm2d-2            [-1, 6, 28, 28]              12
              Tanh-3            [-1, 6, 28, 28]               0
         AvgPool2d-4            [-1, 6, 14, 14]               0
            Conv2d-5           [-1, 16, 10, 10]           2,416
       BatchNorm2d-6           [-1, 16, 10, 10]              32
              Tanh-7           [-1, 16, 10, 10]               0
         AvgPool2d-8             [-1, 16, 5, 5]               0
            Conv2d-9            [-1, 120, 1, 1]          48,120
      BatchNorm2d-10            [-1, 120, 1, 1]             240
             Tanh-11            [-1, 120, 1, 1]               0
          Flatten-12                  [-1, 120]               0
           Linear-13                   [-1, 84]          10,164
      BatchNorm1d-14                   

26427392it [00:01, 13848486.19it/s]                             


Extracting /root/.pytorch/datasets/fashion-mnist/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/datasets/fashion-mnist/FashionMNIST/raw


0it [00:00, ?it/s]

...

4423680it [00:01, 3934115.14it/s]                             
0it [00:00, ?it/s]

Extracting /root/.pytorch/datasets/fashion-mnist/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/datasets/fashion-mnist/FashionMNIST/raw


8192it [00:00, 31249.79it/s]            

Extracting /root/.pytorch/datasets/fashion-mnist/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/datasets/fashion-mnist/FashionMNIST/raw
Processing...
Done!
training on cuda


epoch 1, train loss 0.4934, train acc 0.818, test loss 0.3813, test acc 0.858, time 11.8 sec
epoch 2, train loss 0.3559, train acc 0.869, test loss 0.3862, test acc 0.852, time 11.6 sec
epoch 3, train loss 0.3248, train acc 0.881, test loss 0.3511, test acc 0.872, time 11.4 sec
epoch 4, train loss 0.3086, train acc 0.887, test loss 0.3199, test acc 0.879, time 11.8 sec
epoch 5, train loss 0.2951, train acc 0.892, test loss 0.3074, test acc 0.887, time 11.8 sec
epoch 6, train loss 0.2870, train acc 0.896, test loss 0.3256, test acc 0.883, time 11.7 sec
epoch 7, train loss 0.2803, train acc 0.896, test loss 0.3193, test acc 0.881, time 11.6 sec
epoch 8, train loss 0.2714, train acc 0.900, test loss 0.2961, test acc 0.896, time 11.9 sec
epoch 9, train loss 0.2690, train acc 0.902, test loss 0.2955, test acc 0.894, time 11.6 sec
epoch 10, train loss 0.2618, train acc 0.904, test loss 0.2764, test acc 0.899, time 12.2 sec
